In [1]:
import pandas as pd
df = pd.read_table('C:/Users/sjmal/OneDrive/Desktop/ML/2021/mlchallenge_set_2021_edited.txt')
valid = pd.read_table('C:/Users/sjmal/OneDrive/Desktop/ML/2021/mlchallenge_set_validation.tsv',sep='\t')

In [ ]:
#create dictionary of groups with list of IDs in that group
dict = {}
total=0
grouped_ids = []
for index,row in valid.iterrows():
    if row['Group'] in dict:
        dict[row['Group']].append(row['ID'])
    else:
        dict[row['Group']] = [row['ID']]
#list of groups with multiple IDs
matches = []
for key,value in dict.items():
    if len(value) > 1:
        for i in value:
            grouped_ids.append(i)
        total+=len(value)
        matches.append(key)
print(len(matches))
print(len(grouped_ids))
#print(grouped_ids)

In [3]:
print(df.head())
print(valid.head())

   Category                                          Main Link  \
0         2  https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYYA...   
1         2  https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPIA...   
2         2  https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flIA...   
3         2  http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcAA...   
4         2  https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scsA...   

                                           All Links  \
0  https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYYA...   
1  https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPIA...   
2  https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flIA...   
3  http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcAA...   
4  https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scsA...   

                                                Tags  ID  
0  (Brand:Shimano,US Shoe Size (Men's):4.5,Modifi...   0  
1  (Color:Gray/White,Country/Region of Manufactur...   1  
2  (Style:Cleats,Color:White Orange,US Shoe Size ...   2  
3  (Width:Medium (D, M),US Siz

In [3]:
#split links into list (for now only does first 100)
i = 0
for link in df['All Links'][0:100]:
    df['All Links'][i] = link.split(';')
    i+=1
print(df['All Links'][0:100])

<ipython-input-3-5e0f35be6386>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['All Links'][i] = link.split(';')


0     [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/iYY...
1     [https://i.ebayimg.com/00/s/MTA1OFgxMTM0/z/KPI...
2     [https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/flI...
3     [http://i.ebayimg.com/00/s/ODAwWDEwNjc=/z/XHcA...
4     [https://i.ebayimg.com/00/s/MTA2N1gxNjAw/z/scs...
                            ...                        
95    [https://i.ebayimg.com/00/s/MTU5OVgxNTgx/z/vMA...
96    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/GoA...
97    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/Hm0...
98    [https://i.ebayimg.com/00/s/MTYwMFgxMjAw/z/VhA...
99    [https://i.ebayimg.com/00/s/MTYwMFgxNjAw/z/n4o...
Name: All Links, Length: 100, dtype: object


In [12]:
test_data = pd.DataFrame()
for i in grouped_ids:
    try:
        test_data.append(df[i:])
    except:
        print(i)
print(test_data.head())

Empty DataFrame
Columns: []
Index: []


In [26]:
print(test_data.iloc[0])

Category                                                     2
Main Link    https://i.ebayimg.com/00/s/NzQ0WDE2MDA=/z/B4wA...
All Links    https://i.ebayimg.com/00/s/NzQ0WDE2MDA=/z/B4wA...
Tags         (Closure:Lace Up,Country/Region of Manufacture...
ID                                                         571
Name: 567, dtype: object


In [33]:
import re
def register_attributes(attribute, all_attributes):
    attribute = re.sub(r'[()]','', attribute)
    attribute = re.split(r',', attribute)
    attribute = [a.split(':') for a in attribute]
    for i, a in enumerate(attribute):
        attribute[i] = [s.strip() for s in a]
        all_attributes.add(attribute[i][0])
    #print(f'atttribute is: {attribute}')
    mapping = {}
    #for i in range(len(attribute) - 1):
    #    if i == len(attribute) - 2:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:]
    #    else:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:-1]
    return(attribute)

def map_attributes(attribute, num_attributes, index_to_attr):
    attribute = re.sub(r'[()]','', attribute)
    attribute = re.split(r',', attribute)
    attribute = [a.split(':') for a in attribute]
    all_attributes_for_row = [None] * num_attributes
    for i, a in enumerate(attribute):
        attribute[i] = [s.strip() for s in a]
        #print(f'index: {attr_to_index[attribute[i][0]]}')
        if len(attribute[i]) > 1:
            all_attributes_for_row[attr_to_index[attribute[i][0]]] = attribute[i][1]
    mapping = {}
    #for i in range(len(attribute) - 1):
    #    if i == len(attribute) - 2:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:]
    #    else:
    #        mapping[attribute[i][-1]] = attribute[i + 1][:-1]
    return all_attributes_for_row

all_attributes = set()
all_maps = []
for index,row in df[0:2000].iterrows():
    register_attributes(row['Tags'], all_attributes)

all_attributes = list(all_attributes)
attr_to_index = {all_attributes[i]: i for i in range(len(all_attributes))}
#print(attr_to_index)
#print(f'numAttributes: {len(all_attributes)}')

for index,row in df[0:2000].iterrows():
    all_maps.append(map_attributes(row['Tags'], len(all_attributes), attr_to_index))
#print(all_maps)


In [34]:
#dataframe with attribute values
categories = pd.DataFrame(all_maps)
#print(categories)

In [35]:
from sklearn.preprocessing import OneHotEncoder
x = OneHotEncoder().fit_transform(categories)
#print(x)
print(type(x))

<class 'scipy.sparse.csr.csr_matrix'>


In [31]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(compute_distances=True,compute_full_tree = True,distance_threshold = .01,n_clusters=None).fit(x.todense())
print(clustering)

AgglomerativeClustering(compute_distances=True, compute_full_tree=True,
                        distance_threshold=0.01, n_clusters=None)


In [40]:
from sklearn.cluster import OPTICS
clustering = OPTICS(max_eps=.005,min_samples=2,cluster_method='dbscan').fit(x.toarray())

In [ ]:
#print(clustering.labels_)
print(clustering.core_distances_[clustering.ordering_])
#print(clustering.distances_)
#print(clustering.n_connected_components_)
print(clustering.cluster_hierarchy_)
print(df[0:1000][clustering.ordering_])
for i in clustering.cluster_hierarchy_:
    print(df[clustering.ordering_][i[0],i[1]+1])

In [41]:
groups = {}
for i, label in enumerate(clustering.labels_):
    if label not in groups:
        groups[label] = []
    groups[label].append(i)
groups = {label: groups[label] for label in groups if len(groups[label]) > 1}
print(groups)
for label in groups:
    print(f'GROUP: {label}')
    for item in groups[label]:
        print(df['Tags'][item])
    print('-----------------')

{-1: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225

In [8]:
%matplotlib qt
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
import numpy as np
from PIL import Image
import requests
from io import BytesIO

def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    #response = requests.get(url)
    #img = Image.open(BytesIO(response.content))
    #link_labels = [df['Tags'][i] for i in clustering.labels_]
    dendrogram(linkage_matrix)#labels = link_labels
plot_dendrogram(clustering)
#plt.set_urls(df['Main Link'][i] for i in clustering.labels_)
plt.show()